# Laad Excel

In [ ]:
import pandas as pd

excel_file = "email-list.xlsx"

df = pd.read_excel(excel_file)
df

# Creeer word bestanden vanuit template

In [ ]:
from docx import Document

def replace_text_in_docx(docx_file, replacements):
    doc = Document(docx_file)
    for paragraph in doc.paragraphs:
        for key, value in replacements.items():
            if key in paragraph.text:
                paragraph.text = paragraph.text.replace(key, value)
    
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for key, value in replacements.items():
                        if key in paragraph.text:
                            paragraph.text = paragraph.text.replace(key, value)
    
    return doc

from pathlib import Path
input_docx_file = "brief-template.docx"
for idx, row in df.iterrows():
    output_docx_file = f"out/word/brief-{idx}.docx"
    Path(output_docx_file).parent.mkdir(parents=True, exist_ok=True)


    replacements = {'{name}': row.naam}

    doc = replace_text_in_docx(input_docx_file, replacements)
    doc.save(output_docx_file)

    print(f"'{output_docx_file}'")


# Converteer Word naar PDF

In [ ]:
# brew install libreoffice

import os
import subprocess

input_dir = "out/word/"
output_dir = "out/pdf/"


def convert_to_pdf(input_file, output_dir):
    try:
        # /opt/homebrew/bin/soffice --headless --convert-to pdf --outdir {output_dir} {input_file}
        subprocess.run(
            [
                "soffice",
                "--convert-to",
                "pdf",
                input_file,
                "--outdir",
                output_dir,
            ],
            check=True,
        )
        print("Conversion successful for:", input_file)
    except subprocess.CalledProcessError as e:
        print("Conversion failed for:", input_file, e)


Path(output_dir).mkdir(parents=True, exist_ok=True)

# List all files in the input directory
pths = [str(p) for p in Path(input_dir).glob("*.docx")]

# Iterate over each file and convert it to PDF
for pth in pths:
    print(pth)
    convert_to_pdf(pth, output_dir)

# E-mail PDF bestanden